# TO-DO
1. (DONE) Make config updates recursive. Currently nested dicts are just overwriting. That is, if two config files have a MODEL keyword, the whole model is being overwritten.
2. switch model to `**kwargs`-based setup so we can list all possibly-relevant ones.
3. more-efficient CSV processing

In [1]:
import os
os.chdir('/home/djliden91/git/projects/numerai')
from src.config import config
import yaml
from pathlib import Path
from src.models import fastai_tabular
import sys
import pandas as pd
import numpy as np
import time
from src.config.config import Config
#from fastai.tabular.all import *
from importlib import import_module
from pathlib import Path

from src.utils.eval import FastSubmission
from src.utils.setup import credential
from src.utils.setup import download_current
from src.utils.setup import process_current
from src.utils.setup import init_numerapi
from src.utils.cross_val import EraCV

from src.utils.eval import save_preds

#import src.models

from src.utils.metrics import sharpe, val_corr

# SET THIS TORCH STUFF IN FASTAI SECTION
# set flags / seeds
import gc
#torch.backends.cudnn.benchmark = True
np.random.seed(1)
#torch.manual_seed(1)
#torch.cuda.manual_seed(1)

/home/djliden91/.virtualenvs/numerai/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
model = "fastai_tabular"
configpath = "./src/config/experiments/config_debug.yaml"
default_config = Path("./src/config/default_config.yaml")
cfg = Config(default_config)
model_cfg = Path(f'./src/models/default_configs/{model}.yaml')
model_cls = model.title().replace("_","")
cfg.update_config(model_cfg)
cfg.update_config(configpath)
ct = time.localtime()
current_time = f'{ct[0]}_{ct[1]}_{ct[2]}_{ct[3]}{ct[4]}'
cfg.update_config({'SYSTEM':{'TIME':current_time}})
print(cfg)


CV:
  GAP: 0
  TRAIN_START: 0
  TRAIN_STOP: null
  VAL_END: 209
  VAL_N_ERAS: 4
  VAL_START: 205
DATA:
  REFRESH: false
  SAVE_PROCESSED_TRAIN: true
EVAL:
  SAVE_PREDS: false
  SUBMIT_PREDS: false
MODEL:
  BATCHNORM_FINAL: true
  BATCH_SIZE: 2048
  DROPOUT_P: 0
  LAYERS:
  - 400
  - 400
  LEARNING_RATE: null
  NAME: Unnamed
  N_EPOCHS: 1
  TYPE: fastai tabular
  USE_BATCHNORM: true
  WEIGHT_DECAY: 0
  Y_RANGE:
  - 0
  - 1
SYSTEM:
  DEBUG: false
  TIME: '2021_3_21_1014'



In [3]:
credential()
napi = init_numerapi()

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!


In [4]:
# 2. Define key paths
round = napi.get_current_round()
#current_file = Path(data_dir/f"numerai_dataset_{round}.zip")
train = Path(f"./input/numerai_dataset_{round}/numerai_training_data.csv")
tourn = Path(f"./input/numerai_dataset_{round}/numerai_tournament_data.csv")
processed = Path('./input/training_processed.csv')
output = Path("./output/")

In [5]:
download_current(napi=napi)
training_data, feature_cols, target_cols = process_current(processed, train, tourn)

The dataset has already been downloaded.
You can re-download it with refresh = True
Loading the processed training data from file



In [6]:
print(cfg.MODEL)
model_cls

BATCHNORM_FINAL: true
BATCH_SIZE: 2048
DROPOUT_P: 0
LAYERS:
- 400
- 400
LEARNING_RATE: null
NAME: Unnamed
N_EPOCHS: 1
TYPE: fastai tabular
USE_BATCHNORM: true
WEIGHT_DECAY: 0
Y_RANGE:
- 0
- 1



'FastaiTabular'

In [7]:
# Load Model
modmod = import_module(f'src.models.{model}')
mod = getattr(modmod, model_cls)(**cfg.MODEL.config)

In [8]:
era_split = EraCV(eras = training_data.era)

In [9]:
era_split

EraCV(last era:212)

In [10]:
train_idx, test_idx = era_split.get_splits(valid_start = 209,
                                           valid_n_eras = cfg.CV.VAL_N_ERAS,
                                           train_start = cfg.CV.TRAIN_START,
                                           train_stop = cfg.CV.TRAIN_STOP)

In [11]:
mod.fit(df = training_data, cont_names = list(feature_cols),
          train_idx = list(train_idx), val_idx = list(test_idx))

epoch,train_loss,valid_loss,spearmanr,time
0,0.349135,0.357738,0.014734,00:41


In [12]:
val_preds = mod.predict(training_data.iloc[test_idx])

prediction = val_preds
target = list(training_data.target.iloc[test_idx])
val_era = list(training_data.era[test_idx])
eval_df = pd.DataFrame({'prediction':prediction,
                        'target':target, 'era':val_era})
eval_df
print(sharpe(eval_df))
print(val_corr(eval_df))

0.8501802522929379
0.014772592710707605


In [13]:
%%script false --no-raise-error
# might be necessary
print("training the model\n")
print(("[N | train --------- | valid ----------- |"
       " corr ------------- | time]"))
start = time.time()
with learn.no_bar():
    learn.fit_one_cycle(n_epoch = cfg.TRAIN.N_EPOCHS,
                        wd = cfg.MODEL.WEIGHT_DECAY)
end = time. time()


In [14]:
save_preds(model=mod, chunksize=1e6,
           pred_path = output/"preds.csv",
           feature_cols = feature_cols, tourn_path=tourn)